In [1]:
from influxdb import DataFrameClient
import numpy as np
import collections
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt



In [2]:
nb_ts=144


def mean_ASNs(start_date,end_date,minutes_per_ts=10):
    global nb_ts
    nb_ts=int(24*60/minutes_per_ts)
    print("connexion to database.....")
    client = DataFrameClient(
            host='127.0.0.1',
            port=8086,
            username='root',
            password='root',
            database='data')
    
    print("query....")
    rq="SELECT mean(size) AS mean_size FROM data.autogen.netflow WHERE time > '"+start_date+ "' AND time < '"+end_date+"' GROUP BY ASNs,time("+str(minutes_per_ts)+"m) FILL(null)"
    
    quer=client.query(rq)
    return quer




def mean_ts(start_date,end_date,minutes_per_ts=10):
    global nb_ts
    nb_ts=int(24*60/minutes_per_ts)
    print("connexion to database.....")
    client = DataFrameClient(
            host='127.0.0.1',
            port=8086,
            username='root',
            password='root',
            database='data')
    
    print("query....")
    rq="SELECT mean(size) AS mean_size FROM data.autogen.netflow WHERE time > '"+start_date+ "' AND time < '"+end_date+"' GROUP BY ASNs,ASNd,time("+str(minutes_per_ts)+"m) FILL(null)"
    
    quer=client.query(rq)
    return quer



def query_to_X_and_indexes(query):
    keys=collections.defaultdict.keys(query)
    
    
    print("creating arrays....")
    for key in keys:
        nb_col=query[key]['mean_size'].size
        break
      
    
    
    X=np.zeros([1,nb_col])
    indexes=np.zeros([1,1])

      
    for key in keys:
        row=np.array(query[key]['mean_size']).reshape(1,nb_col)
        X=np.vstack((X,row))
        indexes=np.vstack((indexes,np.array(str(key)).reshape(1,1)))

    X=X[1:]
    indexes=indexes[1:]
    return X,indexes
    


def from_X_to_dataframe(X):
    data=pd.DataFrame(data=X,index=None)

    # Taking care of missing data
    print("Taking care of missing data....")
    data =data.fillna(value=0)
    return data

def Kmeans_model(data):
    # Fitting K-Means to the dataset
    print("clustering....")
    kmeans = KMeans(n_clusters = 1000, init = 'k-means++', random_state = 42)
    return kmeans
    

def y_and_Index(y,indexes):
    nb_rows=y.size
    result=np.hstack((indexes,y.reshape(nb_rows,1)))
    return result

    
    

def diff_table(indexes,X,indexes2,X2):
    commun_keys=set(indexes.flat)&set(indexes2.flat)

    where_are_NaNs = np.isnan(X)
    X[where_are_NaNs] = 0
    
    where_are_NaNs = np.isnan(X2)
    X2[where_are_NaNs] = 0
    
    x=np.hstack((indexes,X))
    x2=np.hstack((indexes2,X2))
    
    diff=np.zeros((len(commun_keys),nb_ts))
    i=0
    
    for key in commun_keys:
        
        a=x[[x[:,0]==key]]
        b=x2[[x2[:,0]==key]]
        a=a[:,1:].astype(float)
        b=b[:,1:].astype(float)
        diff[i,:]=(a-b)/np.sqrt(a*a+b*b)
        i=i+1
        
    where_are_NaNs = np.isnan(diff)
    diff[where_are_NaNs] = 0
    return diff

def mean_plot(diff):
    global nb_ts
    diff_mean=np.mean(diff, axis=0)
    
    plt.scatter(range(0, nb_ts), diff_mean)
    plt.show()

def mean_24h(X,nb_jour):
    
    X_mean=np.zeros((X.shape[0],nb_ts))
    for i in range(1,nb_jour):
        X_mean=X_mean+X[:,0+i*(nb_ts):nb_ts+i*(nb_ts)]
        
    X_mean=X_mean/nb_jour
    return X_mean  




In [ ]:

#####################TRAINING##################################################

quer= mean_ts("2016-08-01T00:00:00.000Z","2016-08-02T00:00:00.000Z")

X,indexes =query_to_X_and_indexes(quer)

data=from_X_to_dataframe(X)

kmeans=Kmeans_model(data)

y = kmeans.fit_predict(data)

result=y_and_Index(y,indexes)

In [ ]:
#########################TESTING###############################################


quer2=mean_ts("2016-08-02T00:00:00.000Z","2016-08-03T00:00:00.000Z")

X2,indexes2 =query_to_X_and_indexes(quer2)

data2=from_X_to_dataframe(X2)

y2 = kmeans.fit_predict(data2)

result2=y_and_Index(y2,indexes2)


In [ ]:
#######################EVALUATION##############################################

diff=diff_table(indexes,X,indexes2,X2)


In [ ]:
###################PLOTTING####################################################

#diff.shape[0]
#one by one

for i in range(1,100):
    plt.scatter(range(0, nb_ts), diff[i,:])
    plt.show()

#mean
mean_plot(diff)

In [ ]:

quer= mean_ASNs("2016-03-21T00:00:00.000Z","2016-03-26T00:00:00.000Z",30)

X,indexes =query_to_X_and_indexes(quer)

X=mean_24h(X,3)


quer= mean_ASNs("2016-03-26T00:00:00.000Z","2016-03-31T00:00:00.000Z",30)

X2,indexes2 =query_to_X_and_indexes(quer)

X2=mean_24h(X2,3)

